### Load API Key

In [1]:
from dotenv import load_dotenv
import os

load_dotenv() 
os.environ["LANGSMITH_TRACING"] = "true"
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

### Define tools

In [2]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1752578404, 'localtime': '2025-07-15 04:20'}, 'current': {'last_updated_epoch': 1752578100, 'last_updated': '2025-07-15 04:15', 'temp_c': 14.4, 'temp_f': 57.9, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 249, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 100, 'feelslike_c': 13.8, 'feelslike_f': 56.9, 'windchill_c': 11.2, 'windchill_f': 52.2, 'heatindex_c': 12.5, 'heatindex_f': 54.4, 'dewpoint_c': 11.8, 'de

### Using Language models

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [4]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [5]:
model_with_tools = model.bind_tools(tools)

In [7]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! It seems like you're starting a conversation. I'm here to help answer your questions. What's on your mind?

Tool calls: []


In [8]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in San Francisco', 'search_depth': 'advanced', 'time_range': 'day'}, 'id': 'qatpwe40w', 'type': 'tool_call'}]


### Creating the agent

In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [10]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hi!


In [13]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (6jfta0qxv)
 Call ID: 6jfta0qxv
  Args:
    query: weather in San Francisco
    search_depth: advanced
    time_range: day
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.sfchronicle.com/weather-forecast/article/bay-area-temperature-shift-winds-20769294.php", "title": "Bay Area in for a cooldown today — except for this region", "content": "Published Time: 2025-07-15T11:00:00Z\n\nBay Area weather: Why this city will buck cooling trend\n\n===============\n\nSkip to main contentSubscribe\n\nBay Area\n\n   San Francisco\n   Transit\n   Crime\n   Drug Crisis\n   Golden Gate P

### Streaming messages

In [14]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (c9xbv57ec)
 Call ID: c9xbv57ec
  Args:
    query: weather in San Francisco
    search_depth: advanced
    time_range: day
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.sfchronicle.com/weather-forecast/article/bay-area-temperature-shift-winds-20769294.php", "title": "Bay Area in for a cooldown today — except for this region", "content": "Published Time: 2025-07-15T11:00:00Z\n\nBay Area weather: Why this city will buck cooling trend\n\n===============\n\nSkip to main contentSubscribe\n\nBay Area\n\n   San Francisco\n   Transit\n   Crime\n   Drug Crisis\n   Golden Gate P

### Streaming tokens

In [15]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

Based| on| the| response| from| the| tool|,| here|'s| a| more| detailed| response|:

|The| weather| in| San| Francisco| today| is| expected| to| be| quite| pleasant|,| with| a| high| of| |72|°F| (|22|°C|)| and| a| low| of| |58|°F| (|14|°C|).| There|'s| a| slight| chance| of| fog| in| the| morning|,| but| it| should| clear| up| later| in| the| day|.| The| rest| of| the| Bay| Area| is| expected| to| experience| a| cooling| trend|,| with| temperatures| ranging| from| the| mid|-|60|s| to| the| mid|-|70|s| (|18|-|24|°C|).

|You| can| check| the| latest| weather| forecast| on| S|FG|ATE| or| SF|Chron|icle| for| more| details|.|

### Adding in memory

In [16]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [18]:
input_message = {"role": "user", "content": "Hi, I'm cattie!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm cattie!
================================== Ai Message ==================================

Hi Cattie! It's nice to meet you! How can I help you today?


In [19]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Cattie!


In [21]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I apologize, but I couldn't find any relevant information about your name. It seems like the previous searches didn't yield any results related to your name. If you could provide more context or clarify what you mean by "What's my name?", I'd be happy to try and help you further!
